In [1]:
from pathlib import Path
import pandas as pd
import altair as alt
import altair_transform
from IPython.display import HTML

In [2]:
def latest_file(path: Path, pattern: str = "*"):
    files = path.glob(pattern)
    return max(files, key=lambda x: x.stat().st_ctime)

In [3]:
particle_path = latest_file(Path('results_csv/'),'particle_results*.csv')
print (f'Particle file used is:   {particle_path.name}')

wafer_path = latest_file(Path('results_csv/'),'wafer_results*.csv')
print (f'Wafer file used is:   {wafer_path.name}')

snips_path = latest_file(Path('results_csv/'),'particle_snips*.csv')
print (f'Snips file used is:   {snips_path.name}')

Particle file used is:   particle_results_07-01-21_03-24.csv
Wafer file used is:   wafer_results_07-01-21_03-24.csv
Snips file used is:   particle_snips_07-01-21_03-24.csv


In [4]:
# Particle data to Long format: Version B
#%%time
particle_results = pd.read_csv(particle_path)
particle_results_matched_only = particle_results.dropna()  # only used for calculating aggregated areas in wafer results
particle_results.rename(columns={'preIndex': '_preIndex',
                                 'wafer': '_wafer',
                                 'polymer': '_polymer',
                                 'treatment': '_treatment',
                                 'postIndex': '_postIndex'}, inplace=True)
pa = pd.DataFrame(particle_results.values, columns=particle_results.columns.str.rsplit('_', 1, expand=True))
# pa.rename(columns={'_area': 'area', '_perimeter': 'perimeter', '_intensities': 'mean_intensity'}, level=0, inplace=True)
#pa.rename(columns={'intensities': 'mean_intensity'}, level=0, inplace=True)

pam = pa.melt(id_vars=[
    ('', 'wafer'), 
    ('', 'polymer'), 
    ('', 'treatment'), 
    ('', 'preIndex'), 
    ('', 'postIndex')], 
                value_vars=[
                    ('area', 'pre'), 
                    ('area', 'post'), 
                    ('perimeter', 'pre'), 
                    ('perimeter', 'post'), 
                    ('intensity', 'pre'), 
                    ('intensity', 'post')])

pam.set_index('variable_1', inplace=True)
postValuesSeries = pam.loc['post', 'value']
pam.drop(index='post', inplace=True)
pam['postValue'] = postValuesSeries.values
pam.reset_index(drop=True, inplace=True)
pam.columns = ['wafer', 'polymer', 'treatment', 'preIndex', 'postIndex', 'prop', 'preValue', 'postValue']

# pam.drop(pam[pam.polymer == 'PVC'].index, inplace=True)

# pam['matched'] = ~pam.preIndex.isna() & ~pam.postIndex.isna()

In [5]:
wafer_results = pd.read_csv(wafer_path)
wafer_results.dropna(inplace=True)  # drop any lines were there were no results

particle_wafer_groups = particle_results_matched_only.groupby('wafer')  # grouping by wafer: only particles that matched between pre and post are used here
for _, group_content in particle_wafer_groups:  # cycle through polymer groups
    pre_area_matched_mean = group_content.area_pre.mean()
    wafer_results.loc[(wafer_results.wafer == group_content.wafer.unique()[0]), 'pre_area_matched'] = round(pre_area_matched_mean)
    post_area_matched_mean = group_content.area_post.mean()
    wafer_results.loc[(wafer_results.wafer == group_content.wafer.unique()[0]), 'post_area_matched'] = round(post_area_matched_mean)



# wafer_results.drop(wafer_results[wafer_results.treatment == 'HCl'].index, inplace=True)  # drop HCl treatment (this was only used for calibration)
# wafer_results.drop(wafer_results[wafer_results.polymer == 'PVC'].index, inplace=True)

wafer_results['particle_loss'] = round((abs(wafer_results.matched_count / wafer_results.pre_count - 1)) * 100) # Get normalised count ratios for each wafer: counts of particles in pre state divided by counts in post state. Subtract 1 to normalise. Take absolute value to treat particle loss and particle addition as the same. Multiply by 100 to get percent values.
wafer_results['area_change'] = round((wafer_results.post_area_matched / wafer_results.pre_area_matched - 1) * 100)  # same for area...

wafer_polymer_groups = wafer_results.groupby('polymer')  # group by polymers

wafer_results_wrangled =pd.DataFrame()  # create empty df to fill in results from loop
for _, group_content in wafer_polymer_groups:  # cycle through polymer groups
        
    group_content['particle_loss_waterCorrected'] = group_content.loc[group_content.treatment != 'water', 'particle_loss'] - group_content.loc[group_content.treatment == 'water', 'particle_loss'].iloc[0]  # subtract count ratio of water from count ratios of all other treatments to get the percentage point error (meaning loss OR addition) of particle numbers caused by each treatment 
    group_content['area_change_waterCorrected'] = group_content.loc[group_content.treatment != 'water', 'area_change'] - group_content.loc[group_content.treatment == 'water', 'area_change'].iloc[0]  # same for area
        
    #group_content.drop(group_content[group_content.treatment == 'water'].index, inplace=True)  # the rows with the water treatments can now be deleted
        
# following line is good example how to not get into "set with copy" warning. Also several bool conditions could be used like that: df.loc[(df['A'] == 'blue') & (df['B'] == 'red') & (df['C'] == 'square'),'D'] = 'M5' (found here: https://stackoverflow.com/questions/21263020/pandas-update-value-if-condition-in-3-columns-are-met)
    group_content.loc[(group_content['particle_loss_waterCorrected'] < 0), 'particle_loss_waterCorrected'] = 0  # set all ratios that were smaller than water ratio (and thus got now negative due to water ratio correction) to 0.
#     group_content.loc[(group_content['area_change_waterCorrected'] > 0), 'area_change_waterCorrected'] = 0  # same for area
    
    wafer_results_wrangled = wafer_results_wrangled.append(group_content)  # save results to df
# melted_wafers = wafer_results_wrangled.melt(id_vars=[
#     'wafer',
#     'polymer',
#     'treatment',
#     'pre_count',
#     'post_count',
#     'matched_count',
#     'pre_area_matched',
#     'post_area_matched'
#     ], value_vars=[
#     'particle_loss', 'area_change', 'particle_loss_waterCorrected', 'area_change_waterCorrected'], var_name='Property', value_name='percent_change')

waterCorr_wafer_results = wafer_results_wrangled.drop(['particle_loss','area_change'], axis=1).copy()
waterCorr_wafer_results.rename(columns={'particle_loss_waterCorrected': 'particle_loss', 'area_change_waterCorrected': 'area_change'}, inplace=True)
waterCorr_wafer_results['mode'] = 'water_corrected'

nonCorr_wafer_results = wafer_results_wrangled.drop(['particle_loss_waterCorrected','area_change_waterCorrected'], axis=1).copy()
nonCorr_wafer_results['mode'] = 'non_corrected'

semiMelted_wafers = pd.concat([waterCorr_wafer_results, nonCorr_wafer_results])

semiMelted_wafers['BDI'] = semiMelted_wafers.pre_background + semiMelted_wafers.post_background + 1 * abs(semiMelted_wafers.pre_background - semiMelted_wafers.post_background)

In [6]:
particle_snips = pd.read_csv(snips_path)

In [12]:
# ===================
# Altair result plots
# ===================

# Preparations
# ------------
alt.data_transformers.disable_max_rows()  # make altair plot DFs with more than 5000 rows


# Interactive selections
# ----------------------
# waferSel= alt.selection_single()
pickedX = alt.selection_single(encodings=['x'], empty='none')  # , on='mouseover', clear='mouseout')  # selection of type "single" allows clicking of one element in one plot to filter other plots
pickedY = alt.selection_single(encodings=['y'], empty='none')  # , on='mouseover', clear='mouseout')  # selection of type "single" allows clicking of one element in one plot to filter other plots
partpic = alt.selection_single(fields=['wafer', 'polymer', 'treatment', 'preIndex', 'postIndex'], empty='none', on='mouseover', clear='mouseout', nearest=True)
modeSelector_radio = alt.binding_radio(options=['water_corrected', 'non_corrected'], name='Heatmap mode:   ')
modeSelector = alt.selection_single(fields=['mode'], bind=modeSelector_radio, init={'mode': 'water_corrected'})
treatSelector = alt.selection_multi(fields=['treatment'], bind='legend')
polSelector = alt.selection_multi(fields=['polymer'], on='mouseover', clear='mouseout', nearest=True)
particleprop_dropdown = alt.binding_select(options=['area', 'perimeter', 'intensity'], name='Particle property')
particleprop_select = alt.selection_single(fields=['prop'], bind=particleprop_dropdown, init={'prop': 'area'})
BDIslider = alt.binding_range(min=semiMelted_wafers.BDI.min(), max=semiMelted_wafers.BDI.max(), step=1, name="BDI ≤")
BDIselector = alt.selection_single(fields=['BDIcutoff'], bind=BDIslider, init={'BDIcutoff': 50})
Nslider = alt.binding_range(min=semiMelted_wafers.pre_count.min(), max=semiMelted_wafers.pre_count.max(), step=1, name="pre_count ≥")
Nselector = alt.selection_single(fields=['Ncutoff'], bind=Nslider, init={'Ncutoff': 50})
prepostSelector_radio = alt.binding_radio(options=['pre_image', 'post_image'], name='Image:   ')
prepostSelector = alt.selection_single(fields=['key'], bind=prepostSelector_radio, init={'key': 'pre_image'})


# Quant Heatmap
# -------------
quantHM = alt.Chart(semiMelted_wafers).mark_rect().encode(
    alt.X('treatment:N', sort=['water', 'H2O2', 'KOH', 'Pentane', 'SPT', 'HCl'], axis=alt.Axis(title=None, orient="top", domain=False)),
    opacity = alt.condition((alt.datum.pre_count >= Nselector.Ncutoff) & (alt.datum.BDI <= BDIselector.BDIcutoff), alt.value(1), alt.value(0.1))
).add_selection(
    modeSelector
).transform_filter(
    modeSelector
).properties(
    height=350, width=180
).add_selection(
    pickedY,
    pickedX
)

countHM = quantHM.encode(
    alt.Y('polymer:N', sort=['ABS', 'EVA', 'LDPE', 'PA6', 'PC', 'PET', 'PMMA', 'PP', 'PS', 'PVC', 'TPU', 'Acrylate', 'Epoxy'], axis=alt.Axis(title=None, orient="left", domain=False)),
    color=alt.Color(
        'particle_loss:Q',
        scale=alt.Scale(scheme='lightmulti', domain=(100, 0), clamp=True),  #color scheme similar to quali heatmap: 'lighttealblue', for +/- changes: 'redblue'
        sort='descending',
        legend=alt.Legend(title=['Loss', '[%]'], gradientLength=300, orient='left', titlePadding=20)),  # , gradientLabelOffset=10)),
    tooltip = alt.Tooltip(['wafer', 'pre_count', 'post_count', 'matched_count', 'particle_loss', 'BDI']),
).properties(
    title='Particle Numbers'
)

areaHM = quantHM.encode(
    alt.Y('polymer:N', sort=['ABS', 'EVA', 'LDPE', 'PA6', 'PC', 'PET', 'PMMA', 'PP', 'PS', 'PVC', 'TPU', 'Acrylate', 'Epoxy'], axis=alt.Axis(title=None, orient="left", domain=False, labels=False, ticks=False)),
    color=alt.Color(
        'area_change:Q',
        scale=alt.Scale(scheme='redblue', domain=(-100, 100), clamp=True),  #color scheme similar to quali heatmap: 'lighttealblue', for +/- changes: 'redblue'
        legend=alt.Legend(title=['Change', '+/- [%]'], gradientLength=300, orient='right', titlePadding=20)),  # , gradientLabelOffset=10)),
    tooltip = alt.Tooltip(['wafer', 'pre_area_matched', 'post_area_matched', 'area_change', 'BDI'])
).properties(
    title='Particle Areas'
)

# tileFrame = quantHM.encode(
#     alt.X('treatment:N', sort=['water', 'H2O2', 'KOH', 'Pentane', 'SPT', 'HCl'], axis=alt.Axis(title=None, orient="top", domain=False), scale=alt.ScaleConfig(bandPaddingInner=0)),
#     alt.Y('polymer:N', sort=['ABS', 'EVA', 'LDPE', 'PA6', 'PC', 'PET', 'PMMA', 'PP', 'PS', 'TPU', 'Acrylate', 'Epoxy'], axis=alt.Axis(title=None, orient="left", domain=False, labels=False, ticks=False), scale=alt.ScaleConfig(bandPaddingInner=0)),
#     color=alt.condition(waferSel, alt.value('black'), alt.value('white'))
# ).add_selection(
#     waferSel
# )


# Wafer scatter
# -------------
waferScatter = alt.Chart(semiMelted_wafers
).mark_point(filled=True).encode(
    alt.X('pre_count'),#, scale=alt.Scale(domain=(0,450))),
    alt.Y(alt.repeat('column'), type='quantitative', sort='ascending'),
    #alt.Y('particle_loss', sort='ascending'),
    color=alt.Color('treatment:N', legend=alt.Legend(orient='bottom')),
    size = alt.Size('BDI:Q', legend=alt.Legend(orient='bottom'), scale = alt.Scale(domain=(semiMelted_wafers.BDI.min()+5, semiMelted_wafers.BDI.max()+5))),
    tooltip=['wafer', 'polymer', 'treatment', 'pre_count', alt.Tooltip(alt.repeat('column'), type='quantitative'), 'BDI:Q'],
    #tooltip=['wafer', 'polymer', 'treatment', 'pre_count', 'particle_loss', 'BDI:Q'],
    opacity=alt.condition(treatSelector | polSelector, alt.value(1), alt.value(0.2))
).properties(
    height=150, width=150
).repeat(
    #row=['pre_count', 'matched_count', 'post_count'],
    column=['particle_loss', 'area_change']
).resolve_scale(
    y='independent'
).add_selection(
    treatSelector
).transform_filter(
    modeSelector
).add_selection(
    BDIselector
).transform_filter(
    alt.datum.BDI <= BDIselector.BDIcutoff
).add_selection(
    Nselector
).transform_filter(
    alt.datum.pre_count >= Nselector.Ncutoff
).add_selection(
    polSelector
).interactive(
)

# waferScatterRegLine = waferScatter.transform_calculate(
#     particle_loss_NZ = 'datum.particle_loss + 0.01'
# ).transform_regression('pre_count', 'particle_loss_NZ', method='linear'
# ).mark_line(clip=True)

# waferScatterRegParams =  waferScatter.transform_calculate(
#     particle_loss_NZ = 'datum.particle_loss + 0.01'
# ).transform_regression('pre_count', 'particle_loss_NZ', method='linear', params=True
# ).mark_text(align='left', lineBreak='\n').encode(
#     x=alt.value(100),  # pixels from left
#     y=alt.value(20),  # pixels from top
#     text='params:N'
# ).transform_calculate(
#     params='"r² = " + round(datum.rSquared * 100)/100 + "      y = " + round(datum.coef[0] * 10)/10 + " * x^" + round(datum.coef[1] * 10)/10'
# )
# #print(altair_transform.extract_data(waferScatterRegParams))

# waferScatter = alt.layer(waferScatter, waferScatterRegLine, waferScatterRegParams
# ).add_selection(
#     treatSelector
# ).transform_filter(
#     modeSelector
# ).add_selection(
#     BDIselector
# ).transform_filter(
#     alt.datum.BDI <= BDIselector.BDIcutoff
# ).add_selection(
#     Nselector
# ).transform_filter(
#     alt.datum.pre_count >= Nselector.Ncutoff
# ).add_selection(
#     polSelector
# ).interactive(
# )


# Wafer images
# ------------
fullImg = alt.Chart(semiMelted_wafers).transform_fold(
    ['pre_image', 'post_image']
).mark_image(
    width=400,
    height=400
).encode(
    url = 'value:N'
).add_selection(
    prepostSelector
).transform_filter(
    prepostSelector
).properties(title = alt.TitleParams('Image', orient='bottom')
)


# Scatter plot
# ------------
particleScatter = alt.Chart(pam).mark_circle(size=100).encode(
    x=alt.X('preValue', axis=alt.Axis(title='Pre value of property')),
    y=alt.Y('postValue', axis=alt.Axis(title='Post value of property')),
    tooltip=alt.Tooltip(['preIndex', 'postIndex', 'preValue', 'postValue'])
).transform_filter(
    pickedX & pickedY
).properties(title = alt.TitleParams('Particles of selcted wafer', orient='top')
)

waterScatter = alt.Chart(pam).mark_circle(color='grey', opacity=0.3).encode(
    x='preValue',
    y='postValue'
).transform_filter(
    {'and': [pickedY, alt.FieldEqualPredicate(field='treatment', equal='water')]}
)

particleLinReg = particleScatter.transform_regression('preValue', 'postValue', method="linear"
).mark_line(color="orange", clip=True)

particleLinRegParams = particleScatter.transform_regression('preValue', 'postValue', method="linear", params=True
).mark_text(align='left', lineBreak='\n').encode(
    x=alt.value(20),  # pixels from left
    y=alt.value(20),  # pixels from top
    text='params:N'
).transform_calculate(
    params='"r² = " + round(datum.rSquared * 100)/100 + "      y = " + round(datum.coef[0] * 10)/10 + " + " + round(datum.coef[1] * 10)/10 + "x"'
)

waterLinReg = waterScatter.transform_regression('preValue', 'postValue',method="linear"
).mark_line(color="grey", clip=True)

# band = alt.Chart(pam).mark_errorband(extent='ci').encode(
#     x='preValue',
#     y='postValue'
# )

identityLine = alt.Chart(pam).mark_line(color= 'black', strokeDash=[3,8], clip=True).encode(
    x=alt.X('preValue', axis=alt.Axis(title='')),
    y=alt.Y('preValue', axis=alt.Axis(title=''))
)

texts = alt.Chart().mark_text(dy=-180, size=12).encode(
    text='label:N'
).transform_calculate(label='datum.treatment + " on " + datum.polymer'
).transform_filter(
    pickedX & pickedY
)


# Boxplot
# -------
boxPlot = alt.Chart(pam).transform_calculate(
    Change='(datum.postValue / datum.preValue -1)'
).mark_boxplot(extent=0.5, outliers=True, clip=True).encode(
    x='treatment',
    y=alt.Y('Change:Q', axis=alt.Axis(format='%'), 
            scale=alt.Scale(domain=(-1, 1)))
).transform_filter(
    particleprop_select
).transform_filter(
    {'and': [pickedY, {'or': [pickedX, alt.FieldEqualPredicate(field='treatment', equal='water')]}]}
)


# Particle snip images
# --------------------
snipPre = alt.Chart().mark_image(
    width=150,
    height=150
).encode(
    url = 'snip_pre:N'
).transform_filter(
    partpic
).properties(title = alt.TitleParams('Pre Treatment', orient='top')
)


snipPost = alt.Chart().mark_image(
    width=150,
    height=150
).encode(
    url = 'snip_post:N'
).transform_filter(
    partpic
).properties(title = alt.TitleParams('Post Treatment', orient='top')
)


# Putting plots together
# ----------------------
# wafer_side = alt.hconcat(tileFrame + countHM, tileFrame + areaHM).resolve_scale(
#     color='independent'
# )

wafer_side = alt.hconcat(countHM, areaHM).resolve_scale(
    color='independent') & waferScatter & fullImg

scatterAll = (particleScatter.add_selection(partpic) + particleLinReg + particleLinRegParams + waterScatter + waterLinReg + identityLine + texts
).add_selection(
    particleprop_select
).transform_filter(
    particleprop_select
).properties(
    width=300,
    height=300
)

snips = alt.hconcat(snipPre, snipPost, data=particle_snips)

particle_side = alt.vconcat(scatterAll | boxPlot, snips, center=True)

display(alt.hconcat(wafer_side, particle_side,
                                padding={"left": 50, "top": 50, "right": 50, "bottom": 50},
                                spacing=50
).configure_scale(bandPaddingInner=0.1  # set space between heatmap tiles
).configure_title(orient='bottom', offset=20  # configure title of plot
).configure_view(
    strokeWidth=0  # get rid of chart box
).save('figures/quant_results.html'
))


None

In [186]:
x = np.arange(1,10)
y = 1 / x -1

df = pd.DataFrame(y, x).reset_index().rename(columns = {'index':'x', 0:'y'})

chart = alt.Chart(df).mark_point().encode(
    x='x',
    y='y'
)

reg = chart.transform_regression('x', 'y', method='pow').mark_line()
regParams = chart.transform_regression('x', 'y', method='pow', params=True).mark_text(align='left', lineBreak='\n').encode(
    x=alt.value(100),  # pixels from left
    y=alt.value(20),  # pixels from top
    text='params:N'
).transform_calculate(
    params='"r² = " + round(datum.rSquared * 100)/100 + "      y = " + round(datum.coef[0] * 10)/10 + " * x^" + round(datum.coef[1] * 10)/10'
)

chart + reg + regParams

alt.LayerChart(...)